# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [3]:
import pandas as pd

In [18]:
# 1. 10 countries with lowest infant mortality rates
cols = ['country', 'infant_mortality']
d = pd.DataFrame(columns=cols)
root = document.getroot()
for child in root:
    if child.tag == 'country':
        if child.find('infant_mortality') is not None:
            infant_mortality = float(child.find('infant_mortality').text)
            country = child.find('name').text
            data = pd.Series([country, infant_mortality], index=cols)
            d = d.append(data, ignore_index=True)
d.sort_values(by='infant_mortality').head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [19]:
# 2. 10 cities with the largest population (using latest population)

root = document.getroot()
city_cols = ['city', 'year', 'population']
city_df = pd.DataFrame(columns=city_cols)
for city in root.iter('city'):
    city_name = city.find('name').text
    max_year = 0
    for population in city.findall('population'):
        test_year = int(population.attrib['year'])
        if test_year > max_year:
            max_year = test_year
            population_num = int(population.text)
    city_s = pd.Series([city_name, max_year, population_num], city_cols)
    city_df = city_df.append(city_s, ignore_index=True)
city_df.sort_values(by="population", ascending=False).head(10)    

,city,year,population
1341,Shanghai,2010.0,22315474.0
771,Istanbul,2012.0,13710512.0
1527,Mumbai,2011.0,12442373.0
479,Moskva,2013.0,11979529.0
1340,Beijing,2010.0,11716620.0
2810,São Paulo,2010.0,11152344.0
1342,Tianjin,2010.0,11090314.0
1064,Guangzhou,2010.0,11071424.0
1582,Delhi,2011.0,11034555.0
1067,Shenzhen,2010.0,10358381.0


In [20]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
eg_cols = ['ethnicgroup', 'population']
eg = pd.DataFrame(columns=eg_cols)

root = document.getroot()
for child in root:
    if (child.tag=='country'):
        #find latest population
        max_year = 0
        for population in child.findall('population'):
            if int(population.attrib['year']) > max_year:
                max_year = population.attrib['year']
                pop_number = int(population.text)
        #find all ethnicgroup, for each ethnicgroup, multiply population with with percentage
        for ethnicgroup in child.findall('ethnicgroup'):
            eg_pop_country = (float(ethnicgroup.attrib['percentage']) * pop_number) / 100.0
            if len(eg[eg.ethnicgroup==ethnicgroup.text]) == 0:
                eg_s = pd.Series([ethnicgroup.text, eg_pop_country], eg_cols)
                eg = eg.append(eg_s, ignore_index=True)
            else:
                eg.population.values[eg.ethnicgroup.values==ethnicgroup.text] += eg_pop_country

eg.sort_values(by="population", ascending=False).head(10)          
        
        

,ethnicgroup,population
80,Han Chinese,4.975551e+08
128,European,1.928658e+08
106,Indo-Aryan,1.716454e+08
33,Russian,9.275844e+07
16,African,8.632937e+07
139,Japanese,8.170627e+07
21,German,6.623219e+07
105,Dravidian,5.959908e+07
70,English,4.231499e+07
150,Mestizo,3.554233e+07


In [39]:
#4. name and country of a) longest river, b) largest lake and c) airport at highest elevation
from IPython.display import display

c_cols = ['country_name', 'country_code']
r_cols = ['river_name', 'river_length', 'country_code']
l_cols = ['lake_name', 'lake_area', 'country_code']
a_cols = ['airport_name', 'airport_elevation', 'country_code']

countries = pd.DataFrame(columns=c_cols)
rivers = pd.DataFrame(columns=r_cols)
lakes = pd.DataFrame(columns=l_cols)
airports = pd.DataFrame(columns=a_cols)

def addtoDataFrame (node, attr_type, attribute, cols, df):
    if (attr_type=='float'):
        format_attr = float(node.find(attribute).text)
    elif (attr_type=='int'):
        format_attr = int(node.find(attribute).text)
    countries = node.attrib['country'].split(' ')
    for country in countries:
        series = pd.Series([node.find('name').text, format_attr, country], cols)
        df = df.append(series, ignore_index=True)
    return df 

root = document.getroot()
for child in root:
    if (child.tag=='country'): 
        country_series = pd.Series([child.find('name').text, child.attrib['car_code']], c_cols)
        countries = countries.append(country_series, ignore_index=True)
    if (child.tag=='river'):
        if child.find('length') is not None:
            rivers = addtoDataFrame(child, 'float', 'length', r_cols, rivers)
    if (child.tag=='lake'):
        if child.find('area') is not None:
            lakes = addtoDataFrame(child, 'float', 'area', l_cols, lakes)
    if (child.tag=='airport'):
        if child.find('elevation') is not None:
            if child.find('elevation').text is not None:
                airports = addtoDataFrame(child, 'float', 'elevation', a_cols, airports)

print 'Longest River:'
river_name = rivers.sort_values(by='river_length', ascending=False).head(1).iloc[0]['river_name']
r = rivers.merge(countries)
display(r[r.river_name==river_name].sort_values(by='river_length', ascending=False))

print 'Largest Lake:'
lake_name = lakes.sort_values(by='lake_area', ascending=False).head(1).iloc[0]['lake_name']
l = lakes.merge(countries)
display(l[l.lake_name==lake_name].sort_values(by='lake_area', ascending=False))

print 'Airport at Highest Elevation:'
display(airports.merge(countries).sort_values(by='airport_elevation', ascending=False).head(1))

Longest River:


,river_name,river_length,country_code,country_name
299,Amazonas,6448.0,CO,Colombia
302,Amazonas,6448.0,BR,Brazil
310,Amazonas,6448.0,PE,Peru


Largest Lake:


,lake_name,lake_area,country_code,country_name
56,Caspian Sea,386400.0,R,Russia
72,Caspian Sea,386400.0,IR,Iran
73,Caspian Sea,386400.0,AZ,Azerbaijan
74,Caspian Sea,386400.0,KAZ,Kazakhstan
77,Caspian Sea,386400.0,TM,Turkmenistan


Airport at Highest Elevation:


,airport_name,airport_elevation,country_code,country_name
80,El Alto Intl,4063.0,BOL,Bolivia
